I stumbled accross another QC metric called SERE. I need to see how it works.

In [1]:
import pandas
import numpy

In [2]:
%load_ext rpy2.ipython

In [19]:
counts = numpy.asarray([
        [1, 1.1, 5  , 3],
        [1,   0, 1.1, 1],
        [5,   7, 5  , 4],
        ])
# counts = pandas.DataFrame(counts, index=['gene1','gene2','gene3'])

totals = counts.sum(axis=0)

In [20]:
totals

array([  7. ,   8.1,  11.1,   8. ])

In [6]:
%%R
SERE_fun <- function(observed, laneTotals, TH=1) {
  #calculate lambda and expected values
  total <- sum(laneTotals)
  fullObserved <- observed[rowSums(observed)>TH,];
  fullLambda <- rowSums(fullObserved)/total;
  fullLhat <- fullLambda > 0;
  fullExpected<- outer(fullLambda, laneTotals);

  #keep values
  fullKeep <- which(rowSums(fullExpected) > 0);
  
  #calculate degrees of freedom (nrow*(ncol -1) >> number of parameters - number calculated parameter (lamda is calculated >> thus minus 1)
  #calculate pearson and deviance for all values
  oeFull <- rowSums((fullObserved[fullKeep,] - fullExpected[fullKeep,])^2/ fullExpected[fullKeep,]) # test for over dispersion
  dfFull <- ncol(observed)*length(fullKeep) - sum(fullLhat!=0);
  devFull <- rowSums( fullObserved[fullKeep,] * log(ifelse(fullObserved[fullKeep,]==0,1,fullObserved[fullKeep,])/fullExpected[fullKeep,]))
  return(sqrt(sum(oeFull)/dfFull));
}


In [21]:
%Rpush counts totals
%R SERE_fun(counts, totals)

array([ 0.74359252])

In [22]:
counts = numpy.asarray([
        [1   , 1.1, 4.9, 3.0],
        [.9  ,   1, 5.1, 3.5],
        [1.01,   3, 5  , 4.0],
        ])
# counts = pandas.DataFrame(counts, index=['gene1','gene2','gene3'])

totals = counts.sum(axis=0)
%Rpush counts totals
%R SERE_fun(counts, totals)

array([ 0.35120678])

In [23]:
counts = numpy.ones(shape=(10000,4))
totals = counts.sum(axis=0)
%Rpush counts totals
%R SERE_fun(counts, totals)

array([ 0.])

In [24]:
noise = numpy.random.normal(loc=0.0, scale=1, size=(10000, 2))
values = numpy.random.lognormal(mean=1.0, sigma=0.5, size=(10000))
counts = pandas.DataFrame(
    [ (v + (n[0]/v), v + (n[1]/v)) for v, n in zip (values, noise) ],
    columns=['rep1', 'rep2'])
%Rpush counts totals
%R SERE_fun(counts, totals)


Error in (fullObserved[fullKeep, ] - fullExpected[fullKeep, ])^2/fullExpected[fullKeep,  : 
  non-conformable arrays


/usr/lib/python3/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Error in (fullObserved[fullKeep, ] - fullExpected[fullKeep, ])^2/fullExpected[fullKeep,  : 
  non-conformable arrays

  res = super(Function, self).__call__(*new_args, **new_kwargs)
